In [1]:
import os, sys, time, glob, random, argparse
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn
from pathlib import Path
sys.path.append(str('/home/jiahzhao/Data/NAS-Projects-master/lib'))
sys.path.append(str('/home/jiahzhao/Data/NAS-Projects-master/configs'))
from config_utils import load_config, dict2config, configure2str
from datasets     import get_datasets, SearchDataset
from procedures   import prepare_seed, prepare_logger, save_checkpoint, copy_checkpoint, get_optim_scheduler
from utils        import get_model_infos, obtain_accuracy
from log_utils    import AverageMeter, time_string, convert_secs2time
from models       import get_cell_based_tiny_net, get_search_spaces
from nas_102_api  import NASBench102API as API

Namespace(all_gpus=False, batch_size=32, bidirectional=True, continue_from=None, cuda=True, data_name='./data_dvae/bench_102_num', data_type='ENAS', epochs=100, hs=501, infer_batch_size=32, keep_old=False, load_latest_model=False, lr=0.0001, model='DVAE', no_cuda=False, no_test=False, nvt=5, nz=30, only_test=False, predictor=False, reprocess=False, sample_number=20, save_appendix='', save_interval=100, seed=1, small_train=False)


In [2]:
import random
from tqdm import tqdm
from shutil import copy
from torch import nn, optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
import scipy.io
from scipy.linalg import qr 
import igraph
from random import shuffle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from data_dvae.util import *
from data_dvae.models_dvae import *
import copy
import argparse

parser = argparse.ArgumentParser("EENAS_V")
parser.add_argument('--data_path',          type=str,   default= '/data/jiahzhao/ENNAS_benchmark102/data/cifar-10-batches-py', help='Path to dataset')
parser.add_argument('--dataset',            type=str,   default= 'cifar10', choices=['cifar10', 'cifar100', 'ImageNet16-120'], help='Choose between Cifar10/100 and ImageNet-16.')
# channels and number-of-cells
parser.add_argument('--search_space_name',  type=str,   default= 'nas-bench-102',help='The search space name.')
parser.add_argument('--max_nodes',          type=int,   default= 4 ,help='The maximum number of nodes.')
parser.add_argument('--channel',            type=int,   default= 16,help='The number of channels.')
parser.add_argument('--num_cells',          type=int,   default= 5, help='The number of cells in one stage.')
parser.add_argument('--track_running_stats',type=int,   default= 1,choices=[0,1],help='Whether use track_running_stats or not in the BN layer.')
parser.add_argument('--config_path',        type=str,   default= '/home/jiahzhao/Data/NAS-Projects-master/configs/nas-benchmark/algos/EENAS.config'  ,help='The path of the configuration.')
# architecture leraning rate
parser.add_argument('--arch_learning_rate', type=float, default=3e-4, help='learning rate for arch encoding')
parser.add_argument('--arch_weight_decay',  type=float, default=1e-3, help='weight decay for arch encoding')
parser.add_argument('--tau_min',            type=float, default=0.1, help='The minimum tau for Gumbel')
parser.add_argument('--tau_max',            type=float, default=10, help='The maximum tau for Gumbel')
# log
parser.add_argument('--workers',            type=int,   default=2,    help='number of data loading workers (default: 2)')
parser.add_argument('--save_dir',           type=str,   default='./EENAS_ratio_10_0.5_seed0/output/search-cell-nas-bench-102-cifar10',help='Folder to save checkpoints and log.')
parser.add_argument('--arch_nas_dataset',   type=str,   default='/home/jiahzhao/Data/NAS-Projects-master/NAS-Bench-102-v1_0-e61699.pth',help='The path to load the architecture dataset (tiny-nas-benchmark).')
parser.add_argument('--print_freq',         type=int,   default=200,help='print frequency (default: 200)')
parser.add_argument('--rand_seed',          type=int,   default=0, help='manual seed')

parser.add_argument('--data-type', default='ENAS',
                    help='DARTS: DARTS-format CNN structures; ENAS: ENAS-format CNN structures; BN: Bayesian networks')
parser.add_argument('--data-name', default='./data_dvae/bench_102_num', help='graph dataset name')
parser.add_argument('--nvt', type=int, default=5, help='number of different node types, \
                    12 for DARTS and 6 for ENAS')
parser.add_argument('--save-appendix', default='', 
                    help='what to append to data-name as save-name for results')
parser.add_argument('--save-interval', type=int, default=100, metavar='N',
                    help='how many epochs to wait each time to save model states')
parser.add_argument('--sample-number', type=int, default=20, metavar='N',
                    help='how many samples to generate each time')
parser.add_argument('--no-test', action='store_true', default=False,
                    help='if True, merge test with train, i.e., no held-out set')
parser.add_argument('--reprocess', action='store_true', default=False,
                    help='if True, reprocess data instead of using prestored .pkl data')
parser.add_argument('--keep-old', action='store_true', default=False,
                    help='if True, do not remove any old data in the result folder')
parser.add_argument('--only-test', action='store_true', default=False,
                    help='if True, perform some experiments without training the model')
parser.add_argument('--small-train', action='store_true', default=False,
                    help='if True, use a smaller version of train set')
# model settings
parser.add_argument('--model', default='DVAE', help='model to use: DVAE, SVAE, \
                    DVAE_fast, DVAE_BN, SVAE_oneshot, DVAE_GCN')
parser.add_argument('--load-latest-model', action='store_true', default=False,
                    help='whether to load latest_model.pth')
parser.add_argument('--continue-from', type=int, default=None, 
                    help="from which epoch's checkpoint to continue training")
parser.add_argument('--hs', type=int, default=501, metavar='N',
                    help='hidden size of GRUs')
parser.add_argument('--nz', type=int, default=30, metavar='N',
                    help='number of dimensions of latent vectors z')
parser.add_argument('--bidirectional', action='store_true', default=True,
                    help='whether to use bidirectional encoding')
parser.add_argument('--predictor', action='store_true', default=False,
                    help='whether to train a performance predictor from latent\
                    encodings and a VAE at the same time')
# optimization settings
parser.add_argument('--lr', type=float, default=1e-4, metavar='LR',
                    help='learning rate (default: 1e-4)')
parser.add_argument('--epochs', type=int, default=100, metavar='N',
                    help='number of epochs to train')
parser.add_argument('--batch-size', type=int, default=32, metavar='N',
                    help='batch size during training')
parser.add_argument('--infer-batch-size', type=int, default=32, metavar='N',
                    help='batch size during inference')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--all-gpus', action='store_true', default=True,
                    help='use all available GPUs')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')

args = parser.parse_args([])


args.cuda = not args.no_cuda and torch.cuda.is_available()
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
np.random.seed(args.seed)
random.seed(args.seed)
print(args)


args.file_dir = os.path.dirname(os.path.realpath('__file__'))
args.res_dir = os.path.join(args.file_dir, 'results/{}{}'.format(args.data_name, 
                                                                 args.save_appendix))

train_data, validation_data,test_data, graph_args = load_bench102_graphs(args.data_name, n_types=args.nvt,fmt='igraph')

model_dvae = eval(args.model)(
        graph_args.max_n, 
        graph_args.num_vertex_type, 
        graph_args.START_TYPE, 
        graph_args.END_TYPE, 
        hs=args.hs, 
        nz=args.nz, 
        bidirectional=args.bidirectional
        )

optimizer = optim.Adam(model_dvae.parameters(), lr=args.lr)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=10, verbose=True)

model_dvae.to(device)

model_dvae.load_state_dict(torch.load('./data_dvae/bench102_model.pt'))  ##################DVAE,,     


model_linear_nn=torch.nn.Sequential(
    torch.nn.Linear(31,100),
    torch.nn.ReLU(),    
    torch.nn.Dropout(0.5),
    torch.nn.Linear(100,100),
    torch.nn.ReLU(),
    torch.nn.Linear(100,args.nz),
     )

####A simple linear model that project vector data to latent representtaion in DVAE
model_linear_nn.load_state_dict(torch.load('./data_dvae/model_linear_nn.pt')) 



pbar_test= tqdm(test_data)
g_test=[]
for i, (g,y) in enumerate(pbar_test):
    g_test.append(g)
mu, logvar= model_dvae.encode(g_test[2])
z = model_dvae.reparameterize(mu, logvar)
m_type=z.cpu()

1023it [00:00, 8545.59it/s]

Namespace(all_gpus=True, arch_learning_rate=0.0003, arch_nas_dataset='/home/jiahzhao/Data/NAS-Projects-master/NAS-Bench-102-v1_0-e61699.pth', arch_weight_decay=0.001, batch_size=32, bidirectional=True, channel=16, config_path='/home/jiahzhao/Data/NAS-Projects-master/configs/nas-benchmark/algos/EENAS.config', continue_from=None, cuda=True, data_name='./data_dvae/bench_102_num', data_path='/data/jiahzhao/ENNAS_benchmark102/data/cifar-10-batches-py', data_type='ENAS', dataset='cifar10', epochs=100, hs=501, infer_batch_size=32, keep_old=False, load_latest_model=False, lr=0.0001, max_nodes=4, model='DVAE', no_cuda=False, no_test=False, num_cells=5, nvt=5, nz=30, only_test=False, predictor=False, print_freq=200, rand_seed=0, reprocess=False, sample_number=20, save_appendix='', save_dir='./EENAS_ratio_10_0.5_seed0/output/search-cell-nas-bench-102-cifar10', save_interval=100, search_space_name='nas-bench-102', seed=1, small_train=False, tau_max=10, tau_min=0.1, track_running_stats=1, workers=2

15625it [00:01, 14764.67it/s]
100%|██████████| 1563/1563 [00:00<00:00, 1519633.09it/s]


# node types: 8
maximum # nodes: 12


In [3]:
from torch.autograd import Variable
from torch.autograd.gradcheck import zero_gradients
import scipy.stats

def update_arch_archive(arch_archive,numb_store):
    arch_archive=arch_archive[:-numb_store]   
    return arch_archive

def cal_novelty(sampled_arch,arch_archive):
    
    def comp_zeros(arch_archive):
        
        arch_archive_t=torch.Tensor(arch_archive)
        app=torch.zeros(arch_archive_t.shape[0],25)
        arch_archive_com=torch.cat((arch_archive_t,app),1)
        
        return arch_archive_com   
    
    def cal_arch_dis(list1,list2):#################small distance more similar
        dis=6
        n_nodes=6######genotypes.STEPS

        for i in range(n_nodes):
            if list1[i]==list1[i]:
                dis=dis-1                   
        dis=dis/6
        return dis 

    def decoder_ori_list(list1):
        z_test_pred = model_linear_nn(list1)    
        uu=z_test_pred-m_type.detach()+m_type
        uu=uu.to(device)
        g_r=model_dvae.decode(uu)
        ff=g_r[0].vs['type']
        ind=[2,4,5,7,8,9]
        decoder_l=[]
        for u in range(6):
            decoder_l.extend([ff[ind[u]]])
        return decoder_l

    
    
    def compute_jacobian(inputs, output):
        """
        :param inputs: Batch X Size (e.g. Depth X Width X Height)
        :param output: Batch X Classes
        :return: jacobian: Batch X Classes X Size
        """
        assert inputs.requires_grad

        num_classes = output.size()[1]

        jacobian = torch.zeros(num_classes, *inputs.size())
        grad_output = torch.zeros(*output.size())
        if inputs.is_cuda:
            grad_output = grad_output.cuda()
            jacobian = jacobian.cuda()

        for i in range(num_classes):
            zero_gradients(inputs)
            grad_output.zero_()
            grad_output[:, i] = 1
            output.backward(grad_output, retain_graph=True)
            jacobian[i] = inputs.grad.data

        return torch.transpose(jacobian, dim0=0, dim1=1)
    
    arch_archive_com=comp_zeros(arch_archive)##########
    archive_f=Variable(arch_archive_com, requires_grad=True)
    archive_z=model_linear_nn(archive_f)
    
    sampled_arch_com=comp_zeros([sampled_arch])##########
    sampled_arch_f=Variable(sampled_arch_com[0], requires_grad=True)
    sampled_arch_z=model_linear_nn(sampled_arch_f)
        
    
    J = compute_jacobian(archive_f, archive_z)
    J = J.cpu().numpy()

    z_size=args.nz#######we set the input dimension as 31, and latent dimensin 30 for easy computation
    train_w=np.array(archive_z.data)
    gennorm_param = np.zeros([3, z_size])######get the latent representation distribution (normal)
    
    for i in range(z_size):
        betta, loc, scale = scipy.stats.gennorm.fit(train_w[:99, i])
        gennorm_param[0, i] = betta
        gennorm_param[1, i] = loc
        gennorm_param[2, i] = scale
    p = scipy.stats.gennorm.pdf(sampled_arch_z.data, gennorm_param[0, :], gennorm_param[1, :], gennorm_param[2, :])
    logPz = np.sum(np.log(p))/30########get the average 
    if not np.isfinite(logPz):
        logPz = -1000
    
    
    u, s, vh = np.linalg.svd(J[0, :, :], full_matrices=False)

    logD = np.sum(np.log(np.abs(s[:6])))########ONLY consider the first 6 dimensions that are non-zero
    

    rlist = []
    for i in range(len(arch_archive)):
        decoder_l=decoder_ori_list(arch_archive_com[i])    
        distance = cal_arch_dis(decoder_l,arch_archive_com[i])
        rlist.append(distance)    
    counts, bin_edges = np.histogram(rlist, bins=6, normed=True)    

    darts_r=decoder_ori_list(arch_archive_com)
    darts_o=arch_archive_com
    distance = cal_arch_dis(darts_r,darts_o)    

    def r_pdf(x, bins, count):
        if x < bins[0]:
            return max(count[0], 1e-308)
        if x >= bins[-1]:
            return max(count[-1], 1e-308)
        id = np.digitize(x, bins) - 1
        return max(count[id], 1e-308)

    logPe = np.log(r_pdf(distance, bin_edges, counts))    
    
    Novelty = (logD + logPz + logPe)/3##########we need to minimize this
    
    return Novelty
    


In [4]:
def cal_compensate(pre_index,cur_index,pre_hardwts,cur_hardwts):
    compen_index   = cur_index
    for i in range(pre_index.shape[0]):
        if pre_index[i][0].item()==cur_index[i][0].item():
            a=np.random.randint(5)
            while a==pre_index[i][0].item():
                a=np.random.randint(5)
            compen_index[i][0]=a              
    one_h   = torch.zeros(6,5).scatter_(-1, compen_index, 1.0).cuda()
    compen_hardwts = one_h - cur_hardwts.detach() + cur_hardwts
    
    return compen_index,compen_hardwts

In [5]:
def sigmoid_function(x):
    return (1/(1+np.exp(-x)))

def search_func(xloader, network, criterion, scheduler, w_optimizer, a_optimizer, epoch_str, print_freq, logger,arch_archive,ex_ep_ratio,multi_model_ratio):
    data_time, batch_time = AverageMeter(), AverageMeter()
    base_losses, base_top1, base_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    arch_losses, arch_top1, arch_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.train()
    end = time.time()

    pre_index=0
    
    for step, (base_inputs, base_targets, arch_inputs, arch_targets) in enumerate(xloader):
      
        scheduler.update(None, 1.0 * step / len(xloader))                   
        base_targets = base_targets.cuda(non_blocking=True)
        arch_targets = arch_targets.cuda(non_blocking=True)
    # measure data loading time
        data_time.update(time.time() - end)    
    # update the weights
        w_optimizer.zero_grad()

######get the current arch loss                
        _, logits,cur_index,cur_hardwts = network(base_inputs,0,0,0)
        base_loss = criterion(logits, base_targets) 
        cur_index,cur_hardwts= network.module.get_index_hardwts()
        
######get the previous arch loss          
        if step==0:
            pre_index=cur_index
            pre_hardwts=cur_hardwts   
            
        _, logits_p,_,_ = network(base_inputs,1,pre_index,pre_hardwts)
        pre_loss = criterion(logits_p, base_targets).item() 
        
######get the compensary arch loss                      
        com_index,com_hardwts=cal_compensate(pre_index,cur_index,pre_hardwts,cur_hardwts)        
        _, logits_c,_,_ = network(base_inputs,1,com_index,com_hardwts)
        compen_loss = criterion(logits_c, base_targets).item()        

      
        eta=1-sigmoid_function(ex_ep_ratio)
        
        lbd=0.5
        
        cur_loss =(1-lbd)*base_loss.item()+lbd/2*(pre_loss+compen_loss)
        cur_loss=base_loss+cur_loss-base_loss.detach()
        
        _, _,cur_index,cur_hardwts = network(base_inputs,0,0,0)########
        cur_loss.backward()
        torch.nn.utils.clip_grad_norm_(network.parameters(), 5)
        w_optimizer.step()
    # record
        base_prec1, base_prec5 = obtain_accuracy(logits.data, base_targets.data, topk=(1, 5))
        base_losses.update(base_loss.item(),  base_inputs.size(0))
        base_top1.update  (base_prec1.item(), base_inputs.size(0))
        base_top5.update  (base_prec5.item(), base_inputs.size(0))

        pre_index=cur_index
        pre_hardwts=cur_hardwts
        
        
        
###########update arch parameters         
        arch_sample=network.module.gene()     
        num_store=50########################################store data   size of A     
        if len(arch_archive)<num_store:
            Novelty=0
        else:
            Novelty=cal_novelty(arch_sample,arch_archive)
                          
    # update the architecture-weight
        a_optimizer.zero_grad()
        _, logits,_,_ = network(arch_inputs,0,0,0)
        a_loss = criterion(logits, arch_targets)
        
        arch_loss=(1-eta)*a_loss+eta*Novelty#####################balance the exploration and exploitation

        arch_loss.backward()
        a_optimizer.step()
    # record
        arch_prec1, arch_prec5 = obtain_accuracy(logits.data, arch_targets.data, topk=(1, 5))
        arch_losses.update(arch_loss.item(),  arch_inputs.size(0))
        arch_top1.update  (arch_prec1.item(), arch_inputs.size(0))
        arch_top5.update  (arch_prec5.item(), arch_inputs.size(0))

    # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if step % print_freq == 0 or step + 1 == len(xloader):
            Sstr = '*SEARCH* ' + time_string() + ' [{:}][{:03d}/{:03d}]'.format(epoch_str, step, len(xloader))
            Tstr = 'Time {batch_time.val:.2f} ({batch_time.avg:.2f}) Data {data_time.val:.2f} ({data_time.avg:.2f})'.format(batch_time=batch_time, data_time=data_time)
            Wstr = 'Base [Loss {loss.val:.3f} ({loss.avg:.3f})  Prec@1 {top1.val:.2f} ({top1.avg:.2f}) Prec@5 {top5.val:.2f} ({top5.avg:.2f})]'.format(loss=base_losses, top1=base_top1, top5=base_top5)
            Astr = 'Arch [Loss {loss.val:.3f} ({loss.avg:.3f})  Prec@1 {top1.val:.2f} ({top1.avg:.2f}) Prec@5 {top5.val:.2f} ({top5.avg:.2f})]'.format(loss=arch_losses, top1=arch_top1, top5=arch_top5)
            logger.log(Sstr + ' ' + Tstr + ' ' + Wstr + ' ' + Astr)
            
            
        arch_archive.append(arch_sample)
        arch_archive=arch_archive[-num_store:]##################consider number of previous
    return base_losses.avg, base_top1.avg, base_top5.avg, arch_losses.avg, arch_top1.avg, arch_top5.avg,arch_archive


In [6]:
#def main(xargs):
xargs=args
assert torch.cuda.is_available(), 'CUDA is not available.'
torch.backends.cudnn.enabled   = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.set_num_threads( xargs.workers )
prepare_seed(xargs.rand_seed)
logger = prepare_logger(args)

train_data, valid_data, xshape, class_num = get_datasets(xargs.dataset, xargs.data_path, -1)
if xargs.dataset == 'cifar10' or xargs.dataset == 'cifar100':
    split_Fpath = '/home/jiahzhao/Data/NAS-Projects-master/configs/nas-benchmark/cifar-split.txt'
    cifar_split = load_config(split_Fpath, None, None)
    train_split, valid_split = cifar_split.train, cifar_split.valid
    logger.log('Load split file from {:}'.format(split_Fpath))
elif xargs.dataset.startswith('ImageNet16'):
    split_Fpath = 'configs/nas-benchmark/{:}-split.txt'.format(xargs.dataset)
    imagenet16_split = load_config(split_Fpath, None, None)
    train_split, valid_split = imagenet16_split.train, imagenet16_split.valid
    logger.log('Load split file from {:}'.format(split_Fpath))
else:
    raise ValueError('invalid dataset : {:}'.format(xargs.dataset))
  #config_path = 'configs/nas-benchmark/algos/GDAS.config'
config = load_config(xargs.config_path, {'class_num': class_num, 'xshape': xshape}, logger)

train_data_v2 = deepcopy(train_data)
train_data_v2.transform = valid_data.transform
valid_data    = train_data_v2
search_data   = SearchDataset(xargs.dataset, train_data, train_split, valid_split)
# data loader
search_loader = torch.utils.data.DataLoader(search_data, batch_size=config.batch_size, shuffle=True , num_workers=xargs.workers, pin_memory=True)
valid_loader  = torch.utils.data.DataLoader(valid_data, batch_size=config.batch_size, sampler=torch.utils.data.sampler.SubsetRandomSampler(valid_split), num_workers=xargs.workers, pin_memory=True)


logger.log('||||||| {:10s} ||||||| Search-Loader-Num={:}, batch size={:}'.format(xargs.dataset, len(search_loader), config.batch_size))
logger.log('||||||| {:10s} ||||||| Config={:}'.format(xargs.dataset, config))

search_space = get_search_spaces('cell', xargs.search_space_name)
model_config = dict2config({'name': 'EENAS', 'C': xargs.channel, 'N': xargs.num_cells,
                          'max_nodes': xargs.max_nodes, 'num_classes': class_num,
                          'space'    : search_space,
                          'affine'   : False, 'track_running_stats': bool(xargs.track_running_stats)}, None)
search_model = get_cell_based_tiny_net(model_config)
#from models.cell_searchs.search_model_gdas import TinyNetworkGDAS as get_cell_based_tiny_net
#search_model = get_cell_based_tiny_net(xargs.channel,xargs.num_cells,xargs.max_nodes,class_num,search_space, False,bool(xargs.track_running_stats))
logger.log('search-model :\n{:}'.format(search_model))

w_optimizer, w_scheduler, criterion = get_optim_scheduler(search_model.get_weights(), config)
a_optimizer = torch.optim.Adam(search_model.get_alphas(), lr=xargs.arch_learning_rate, betas=(0.5, 0.999), weight_decay=xargs.arch_weight_decay)
logger.log('w-optimizer : {:}'.format(w_optimizer))
logger.log('a-optimizer : {:}'.format(a_optimizer))
logger.log('w-scheduler : {:}'.format(w_scheduler))
logger.log('criterion   : {:}'.format(criterion))
#flop, param  = get_model_infos(search_model, xshape)
#logger.log('{:}'.format(search_model))
#logger.log('FLOP = {:.2f} M, Params = {:.2f} MB'.format(flop, param))
logger.log('search-space : {:}'.format(search_space))
if xargs.arch_nas_dataset is None:
    api = None
else:
    api = API(xargs.arch_nas_dataset)
logger.log('{:} create API = {:} done'.format(time_string(), api))

last_info, model_base_path, model_best_path = logger.path('info'), logger.path('model'), logger.path('best')
network, criterion = torch.nn.DataParallel(search_model,device_ids=[0]).cuda(), criterion.cuda()

if last_info.exists(): # automatically resume from previous checkpoint
    logger.log("=> loading checkpoint of the last-info '{:}' start".format(last_info))
    last_info   = torch.load(last_info)
    start_epoch = last_info['epoch']
    checkpoint  = torch.load(last_info['last_checkpoint'])
    genotypes   = checkpoint['genotypes']
    valid_accuracies = checkpoint['valid_accuracies']
    search_model.load_state_dict( checkpoint['search_model'] )
    w_scheduler.load_state_dict ( checkpoint['w_scheduler'] )
    w_optimizer.load_state_dict ( checkpoint['w_optimizer'] )
    a_optimizer.load_state_dict ( checkpoint['a_optimizer'] )
    logger.log("=> loading checkpoint of the last-info '{:}' start with {:}-th epoch.".format(last_info, start_epoch))
else:
    logger.log("=> do not find the last-info file : {:}".format(last_info))
    start_epoch, valid_accuracies, genotypes = 0, {'best': -1}, {}

  # start training
start_time, search_time, epoch_time, total_epoch = time.time(), AverageMeter(), AverageMeter(), config.epochs + config.warmup
arch_archive=[]
for epoch in range(start_epoch, total_epoch):
    w_scheduler.update(epoch, 0.0)
    need_time = 'Time Left: {:}'.format( convert_secs2time(epoch_time.val * (total_epoch-epoch), True) )
    epoch_str = '{:03d}-{:03d}'.format(epoch, total_epoch)
    search_model.set_tau( xargs.tau_max - (xargs.tau_max-xargs.tau_min) * epoch / (total_epoch-1) )
    logger.log('\n[Search the {:}-th epoch] {:}, tau={:}, LR={:}'.format(epoch_str, need_time, search_model.get_tau(), min(w_scheduler.get_lr())))
    
    exp_ratio=10########################define the exporation ratio
    forg_ratio=0.5##########################################define the forgetting ratio
    ex_ep_ratio=((epoch/total_epoch)*2-1)*exp_ratio

    multi_model_ratio=0.5
    
    
    search_w_loss, search_w_top1, search_w_top5, valid_a_loss , valid_a_top1 , valid_a_top5, arch_archive \
              = search_func(search_loader, network, criterion, w_scheduler, w_optimizer, a_optimizer, epoch_str, xargs.print_freq, logger,arch_archive,ex_ep_ratio,multi_model_ratio)
    search_time.update(time.time() - start_time)
    logger.log('[{:}] searching : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%, time-cost={:.1f} s'.format(epoch_str, search_w_loss, search_w_top1, search_w_top5, search_time.sum))
    logger.log('[{:}] evaluate  : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%'.format(epoch_str, valid_a_loss , valid_a_top1 , valid_a_top5 ))
    # check the best accuracy
    valid_accuracies[epoch] = valid_a_top1
    if valid_a_top1 > valid_accuracies['best']:
        valid_accuracies['best'] = valid_a_top1
        genotypes['best']        = search_model.genotype()
        find_best = True
    else: find_best = False

    genotypes[epoch] = search_model.genotype()
    logger.log('<<<--->>> The {:}-th epoch : {:}'.format(epoch_str, genotypes[epoch]))
    # save checkpoint
    save_path = save_checkpoint({'epoch' : epoch + 1,
                'args'  : deepcopy(xargs),
                'search_model': search_model.state_dict(),
                'w_optimizer' : w_optimizer.state_dict(),
                'a_optimizer' : a_optimizer.state_dict(),
                'w_scheduler' : w_scheduler.state_dict(),
                'genotypes'   : genotypes,
                'valid_accuracies' : valid_accuracies},
                model_base_path, logger)
    last_info = save_checkpoint({
          'epoch': epoch + 1,
          'args' : deepcopy(args),
          'last_checkpoint': save_path,
          }, logger.path('info'), logger)
    if find_best:
        logger.log('<<<--->>> The {:}-th epoch : find the highest validation accuracy : {:.2f}%.'.format(epoch_str, valid_a_top1))
        copy_checkpoint(model_base_path, model_best_path, logger)
    with torch.no_grad():
        logger.log('arch-parameters :\n{:}'.format( nn.functional.softmax(search_model.arch_parameters, dim=-1).cpu() ))
    if api is not None: logger.log('{:}'.format(api.query_by_arch( genotypes[epoch] )))
    # measure elapsed time
    epoch_time.update(time.time() - start_time)
    start_time = time.time()

logger.log('\n' + '-'*100)
  # check the performance from the architecture dataset
logger.log('EENAS : run {:} epochs, cost {:.1f} s, last-geno is {:}.'.format(total_epoch, search_time.sum, genotypes[total_epoch-1]))
if api is not None: logger.log('{:}'.format( api.query_by_arch(genotypes[total_epoch-1]) ))
logger.close()

Main Function with logger : Logger(dir=EENAS_ratio_10_0.5_seed0/output/search-cell-nas-bench-102-cifar10, use-tf=False, writer=None)
Arguments : -------------------------------
all_gpus         : True
arch_learning_rate : 0.0003
arch_nas_dataset : /home/jiahzhao/Data/NAS-Projects-master/NAS-Bench-102-v1_0-e61699.pth
arch_weight_decay : 0.001
batch_size       : 32
bidirectional    : True
channel          : 16
config_path      : /home/jiahzhao/Data/NAS-Projects-master/configs/nas-benchmark/algos/EENAS.config
continue_from    : None
cuda             : True
data_name        : ./data_dvae/bench_102_num
data_path        : /data/jiahzhao/ENNAS_benchmark102/data/cifar-10-batches-py
data_type        : ENAS
dataset          : cifar10
epochs           : 100
file_dir         : /data/jiahzhao/NAS-Projects-master/exps/algos
hs               : 501
infer_batch_size : 32
keep_old         : False
load_latest_model : False
lr               : 0.0001
max_nodes        : 4
model            : DVAE
no_cuda    

[2020-01-27 12:38:21] create API = NASBench102API(15625/15625 architectures) done
=> loading checkpoint of the last-info 'EENAS_ratio_10_0.5_seed0/output/search-cell-nas-bench-102-cifar10/seed-0-last-info.pth' start
=> loading checkpoint of the last-info '{'epoch': 237, 'args': Namespace(all_gpus=True, arch_learning_rate=0.0003, arch_nas_dataset='/home/jiahzhao/Data/NAS-Projects-master/NAS-Bench-102-v1_0-e61699.pth', arch_weight_decay=0.001, batch_size=32, bidirectional=True, channel=16, config_path='/home/jiahzhao/Data/NAS-Projects-master/configs/nas-benchmark/algos/EENAS.config', continue_from=None, cuda=True, data_name='./data_dvae/bench_102_num', data_path='/data/jiahzhao/ENNAS_benchmark102/data/cifar-10-batches-py', data_type='ENAS', dataset='cifar10', epochs=100, file_dir='/data/jiahzhao/NAS-Projects-master/exps/algos', hs=501, infer_batch_size=32, keep_old=False, load_latest_model=False, lr=0.0001, max_nodes=4, model='DVAE', no_cuda=False, no_test=False, num_cells=5, nvt=5, nz=3

/data/jiahzhao/anaconda3/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:6290: RuntimeWarning: overflow encountered in power
  return np.log(0.5*beta) - sc.gammaln(1.0/beta) - abs(x)**beta
/data/jiahzhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:90: RuntimeWarning: divide by zero encountered in log


*SEARCH* [2020-01-27 12:44:49] [237-250][200/391] Time 2.10 (1.93) Data 0.00 (0.00) Base [Loss 0.475 (0.504)  Prec@1 87.50 (82.48) Prec@5 100.00 (99.02)] Arch [Loss 0.592 (0.614)  Prec@1 84.38 (78.53) Prec@5 98.44 (98.62)]
*SEARCH* [2020-01-27 12:51:52] [237-250][390/391] Time 2.11 (2.07) Data 0.00 (0.00) Base [Loss 0.444 (0.490)  Prec@1 85.00 (82.95) Prec@5 100.00 (99.11)] Arch [Loss 1.029 (0.617)  Prec@1 67.50 (78.53) Prec@5 97.50 (98.67)]
[237-250] searching : loss=0.49, accuracy@1=82.95%, accuracy@5=99.11%, time-cost=810.3 s
[237-250] evaluate  : loss=0.62, accuracy@1=78.53%, accuracy@5=98.67%
<<<--->>> The 237-250-th epoch : Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_1x1~0|nor_conv_3x3~1|+|nor_conv_1x1~0|nor_conv_3x3~1|nor_conv_3x3~2|)
Find EENAS_ratio_10_0.5_seed0/output/search-cell-nas-bench-102-cifar10/checkpoint/seed-0-basic.pth exist, delete is at first before saving
save checkpoint into EENAS_ratio_10_0.5_seed0/output/search-cell-nas-bench-102-cifar10/checkpoint/seed-

*SEARCH* [2020-01-27 13:28:52] [240-250][200/391] Time 2.32 (2.23) Data 0.00 (0.00) Base [Loss 0.373 (0.479)  Prec@1 89.06 (83.61) Prec@5 98.44 (99.19)] Arch [Loss 0.630 (0.614)  Prec@1 78.12 (78.73) Prec@5 100.00 (98.80)]
*SEARCH* [2020-01-27 13:36:00] [240-250][390/391] Time 2.15 (2.24) Data 0.00 (0.00) Base [Loss 0.920 (0.476)  Prec@1 65.00 (83.67) Prec@5 97.50 (99.18)] Arch [Loss 0.687 (0.612)  Prec@1 80.00 (78.90) Prec@5 100.00 (98.74)]
[240-250] searching : loss=0.48, accuracy@1=83.67%, accuracy@5=99.18%, time-cost=3456.9 s
[240-250] evaluate  : loss=0.61, accuracy@1=78.90%, accuracy@5=98.74%
<<<--->>> The 240-250-th epoch : Structure(4 nodes with |nor_conv_1x1~0|+|nor_conv_3x3~0|avg_pool_3x3~1|+|skip_connect~0|nor_conv_3x3~1|nor_conv_3x3~2|)
Find EENAS_ratio_10_0.5_seed0/output/search-cell-nas-bench-102-cifar10/checkpoint/seed-0-basic.pth exist, delete is at first before saving
save checkpoint into EENAS_ratio_10_0.5_seed0/output/search-cell-nas-bench-102-cifar10/checkpoint/seed

*SEARCH* [2020-01-27 14:13:45] [243-250][200/391] Time 2.37 (2.32) Data 0.00 (0.00) Base [Loss 0.223 (0.432)  Prec@1 93.75 (84.91) Prec@5 100.00 (99.16)] Arch [Loss 0.541 (0.604)  Prec@1 84.38 (79.37) Prec@5 98.44 (98.75)]
*SEARCH* [2020-01-27 14:21:12] [243-250][390/391] Time 2.23 (2.33) Data 0.00 (0.00) Base [Loss 0.515 (0.453)  Prec@1 85.00 (84.32) Prec@5 97.50 (99.15)] Arch [Loss 0.513 (0.613)  Prec@1 80.00 (78.88) Prec@5 100.00 (98.66)]
[243-250] searching : loss=0.45, accuracy@1=84.32%, accuracy@5=99.15%, time-cost=6168.3 s
[243-250] evaluate  : loss=0.61, accuracy@1=78.88%, accuracy@5=98.66%
<<<--->>> The 243-250-th epoch : Structure(4 nodes with |nor_conv_1x1~0|+|nor_conv_3x3~0|nor_conv_3x3~1|+|skip_connect~0|nor_conv_3x3~1|nor_conv_3x3~2|)
Find EENAS_ratio_10_0.5_seed0/output/search-cell-nas-bench-102-cifar10/checkpoint/seed-0-basic.pth exist, delete is at first before saving
save checkpoint into EENAS_ratio_10_0.5_seed0/output/search-cell-nas-bench-102-cifar10/checkpoint/seed

*SEARCH* [2020-01-27 14:59:09] [246-250][200/391] Time 2.33 (2.31) Data 0.00 (0.00) Base [Loss 0.365 (0.480)  Prec@1 89.06 (83.40) Prec@5 100.00 (99.07)] Arch [Loss 0.845 (0.651)  Prec@1 70.31 (78.08) Prec@5 98.44 (98.38)]
*SEARCH* [2020-01-27 15:06:22] [246-250][390/391] Time 2.60 (2.29) Data 0.00 (0.00) Base [Loss 0.819 (0.495)  Prec@1 67.50 (82.87) Prec@5 97.50 (98.99)] Arch [Loss 0.513 (0.652)  Prec@1 85.00 (78.01) Prec@5 100.00 (98.27)]
[246-250] searching : loss=0.49, accuracy@1=82.87%, accuracy@5=98.99%, time-cost=8877.2 s
[246-250] evaluate  : loss=0.65, accuracy@1=78.01%, accuracy@5=98.27%
<<<--->>> The 246-250-th epoch : Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_1x1~0|nor_conv_3x3~1|+|skip_connect~0|nor_conv_3x3~1|nor_conv_3x3~2|)
Find EENAS_ratio_10_0.5_seed0/output/search-cell-nas-bench-102-cifar10/checkpoint/seed-0-basic.pth exist, delete is at first before saving
save checkpoint into EENAS_ratio_10_0.5_seed0/output/search-cell-nas-bench-102-cifar10/checkpoint/seed

*SEARCH* [2020-01-27 15:41:15] [249-250][200/391] Time 1.87 (1.83) Data 0.00 (0.00) Base [Loss 0.195 (0.427)  Prec@1 93.75 (85.12) Prec@5 100.00 (99.28)] Arch [Loss 0.300 (0.609)  Prec@1 87.50 (79.52) Prec@5 100.00 (98.71)]
*SEARCH* [2020-01-27 15:47:06] [249-250][390/391] Time 1.81 (1.84) Data 0.00 (0.00) Base [Loss 0.389 (0.435)  Prec@1 92.50 (84.98) Prec@5 100.00 (99.17)] Arch [Loss 0.351 (0.591)  Prec@1 90.00 (80.26) Prec@5 97.50 (98.71)]
[249-250] searching : loss=0.43, accuracy@1=84.98%, accuracy@5=99.17%, time-cost=11321.2 s
[249-250] evaluate  : loss=0.59, accuracy@1=80.26%, accuracy@5=98.71%
<<<--->>> The 249-250-th epoch : Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_1x1~0|nor_conv_3x3~1|+|skip_connect~0|nor_conv_3x3~1|nor_conv_3x3~2|)
Find EENAS_ratio_10_0.5_seed0/output/search-cell-nas-bench-102-cifar10/checkpoint/seed-0-basic.pth exist, delete is at first before saving
save checkpoint into EENAS_ratio_10_0.5_seed0/output/search-cell-nas-bench-102-cifar10/checkpoint/se